In [33]:
import pandas as pd
import numpy as np
import scipy.stats
import sklearn
import scipy
import sklearn.datasets
import matplotlib.pyplot as plt
import random
%matplotlib inline

Тот же регрессор, который и был в прошлой задаче

In [34]:
class LinearRegressor():
    weights = []
    def __init__(self):
        self.weights = []
    def fit(self, X, y):
        self.weights = scipy.linalg.solve(np.dot(np.transpose(X), X), np.dot(np.transpose(X), y))
    def predict(self, X):
        return np.dot(X, self.weights)

Пусть $X_i =β_1+iβ_2+\epsilon_0+...+\epsilon_i,i=0,1,...,n $—расстояния,котороепроехалтрамвай за i секунд по показанию датчика. Здесь β1 — начальное расстояние, β2 — скорость трамвая, $\epsilon_0$ — ошибка начального показания датчика. Трамвай едет с постоянной скоростью, и через каждую секунду датчик фиксирует расстояние, которое проехал трамвай. Отсчет времени идет от предыдущего замера, причем отсчет происходит с ошибкой. Для i = 1, ..., n величина $\epsilon_i$ есть ошибка приращения расстояния, то есть $\epsilon_i = \epsilon^t_i β_2, где \epsilon^t_i$ — ошибка отсчета времени. Все ошибки $\epsilon_i$ независимы и распределены по закону$ N(0, \sigma^2)$. Сведите задачу к линейной модели и найдите оценки наименьших квадратов для начального расстояния $β_1$ и скорости$ β_2$, а также несмещенную оценку для $\sigma^2$, из которой выразите оценку дисперсии отсчета времени. Сделайте выводы.

In [35]:
distance = pd.read_csv('/Users/semenfedotov/Downloads/497 Федотов Семен.txt', names=['distance'])

In [36]:
distance.head(3)

,distance
0,38.822031
1,42.249063
2,46.398531


In [37]:
distance.shape

(61, 1)

Возьмем такие случайные величины $Y_i = X_i - X_{i - 1}, \forall i \neq 0, Y_0 = X_0 = \beta_1 + \epsilon_0$

Тогда $Y_i = \beta_2 + \epsilon_i$
Получается, мы свели к регрессионной модели. мы можем применить наши знания об оценивании параметров beta2


In [38]:
selection = [distance['distance'][i] - distance['distance'][i - 1] if i != 0 else distance['distance'][0] for i in range(distance.shape[0])]

In [39]:
selection = pd.DataFrame(selection, columns=['distance'])

In [40]:
selection.head(3)

,distance
0,38.822031
1,3.427032
2,4.149468


In [41]:
selection_size = selection.shape[0]

На данный момент мы свели задачу к линейной регрессионой модели
разложим теперь каждый элемент выборки по базису $\beta_1, \beta_2$ 
Получим такую матрицу

In [42]:
measures = np.matrix([[1] + [0] * (selection_size - 1),[0] + [1] * (selection_size - 1)])

In [43]:
measures = pd.DataFrame(measures.transpose(), columns=['beta_1', 'beta_2'])

In [44]:
measures.head(3)

,beta_1,beta_2
0,1,0
1,0,1
2,0,1


Теперь найдем оптимальную оценку для $\beta_1, \beta_2$
$$(\beta_1, \beta_2) = {(X^TX)}^{-1}X^Ty$$



In [45]:
evaluator = LinearRegressor()
evaluator.fit(measures.as_matrix(), selection)

In [46]:
betas = evaluator.weights

Вот веса нашего регрессора, которые и являются оценками наших бет. оценка beta_1 = selection[0] Т.е $Y_0$

In [47]:
betas

array([[ 38.822031  ],
       [  3.86509417]])

Как мы знаем, несмещенной оценкой параметра $\sigma^2$ в линейной модели явялется $\frac{1}{n - k}||X − Z\theta||^2,$ где $\theta$ вектор весов(оценок параметров бета1 и бета2).
В нашем случае n = 61, k = 2

In [48]:
sigma_squared = (1. / (61. - 2.)) * np.dot((selection.as_matrix() - np.dot(measures.as_matrix(), betas)).transpose(),\
                                           (selection.as_matrix() - np.dot(measures.as_matrix(), betas)))

Нашли неспещенную оценку параметра $\sigma^2$

In [49]:
sigma_squared = sigma_squared[0][0]

In [50]:
sigma_squared

0.27111391664787132

 выразите оценку дисперсии отсчета времени. Сделайте выводы. так как произведение имеет нормальное распределение, а один из множителей константа, то , поделив на него, у нас дисперсия изменится в квадрат раз, так и получим:

In [51]:
sigma_squared_for_time = sigma_squared / (betas[1][0] ** 2)

In [52]:
sigma_squared_for_time

0.018148120430563176

In [54]:
ans = [betas[0][0], betas[1][0], sigma_squared, sigma_squared_for_time]

In [55]:
print ans

[38.822031000000003, 3.8650941666666672, 0.27111391664787132, 0.018148120430563176]


Мы получили , что ошибка при мерениях очень маленькая(близка к нулю). Т.е оценки , полученные нами, должны быть очень близкими к реальным данным